# Keras Tuner 简介

本页内容
- 概述
- 设置
- 下载并准备数据集
- 定义模型
- 实例化调节器并执行超调
- 训练模型
- 总结


## 概述

Keras tuner是一个库，可以帮助你为Tensorflow程序选择最佳的超参数集。为您的机器学习（ML）应用选择正确的超参数集，这一过程称为超参数调节或超调

超参数是控制训练过程和机器学习模型的拓扑的变量，这些变量在训练过程汇总保持不变，并会直接影响机器学习程序的性能，超参数有两种类型：

- 模型超参数：影响模型的选择，例如隐藏层的数量和宽度
- 算法超参数：影响学习算法的速度和质量，例如随机梯度下降（SGD）的学习率以及K紧邻（KNN）分类器的近邻数




In [9]:
import keras
# SECTION: 设置

import tensorflow as tf

import keras_tuner as kt

print(tf.__version__)

2.13.0


## 下载并准备数据集

在本教程中，您将使用 Keras Tuner 为某个对 Fashion MNIST 数据集内的服装图像进行分类的机器学习模型找到最佳超参数。

In [10]:
# SECTION： 加载数据

(img_train, label_train), (img_test, label_test) = tf.keras.datasets.fashion_mnist.load_data()

# image的数字标准化可以提高后续训练模型的准确率，因为image的数字是从0到255的值，所以最简单的标准化方式是除以255
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

## 定义模型
构建用于超调的模型时，除了模型架构之外，还要定义超参数搜索空间，您为超调设置的模型称为超模型

您可以通过两种方式定义超模型

- 使用模型构建攻击函数
- 将Keras Tuner API的HyperModel类子类化

在本教程中，您将使用模型构建工具函数来定义图像分类模型。模型构建工具函数将返回已编译的模型，并使用您以内嵌方式定义的超参数对模型进行超调。

In [11]:
def model_builder(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))

    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(tf.keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(tf.keras.layers.Dense(10))

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    # 返回一个构建好的模型（compiled model，指整体模型结构已经搭好，包括损失函数的定义等等，但模型还未经过训练）：
    return model


## 实例化调节器并执行超调

实例化调节器以执行超调，Keras Tuner 提供了四种调节器：RandomSearch、Hyperband、BayesianOptimaizaation和Sklearn,您将使用Hyperband调机器

要实例化Hyperband调节器，必须指定超模型，要优化的objective和要训练的最大周期数（max_epochs）

In [12]:
tuner = kt.Hyperband(model_builder,
            objective='val_accuracy',
            max_epochs=10,
            factor=3,
            directory='my_dir',
            project_name='intro_to_kt'
)

Hyperband调节算法使用自适应资源分配和早停法来快速收敛到高性能模型，该过程才用体育经济争冠模式的排除法。算法会将大量模型训练多个周期，并仅将性能最高的一半模型送入到下一轮训练，Hyperband通过计算1 + log（factor(max_epochs)）并将其向上舍入到最近的整数来确定要训练的模型的数量

创建回调以在验证损失达到特定值后提前停止训练

In [13]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

运行超参数搜索，除了上面的回调外， 搜索方法的参数也与tf.keras.model.fit所用参数相同

In [15]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2,
 callbacks=[stop_early])

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 26 Complete [00h 01m 24s]
val_accuracy: 0.8922500014305115

Best val_accuracy So Far: 0.8922500014305115
Total elapsed time: 00h 09m 08s

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 448 and the optimal learning rate for the optimizer
is 0.001.



## 训练模型

使用从搜索中获得的超参数找到训练模型的最佳周期数

In [16]:
model = tuner.hypermodel.build(best_hps)

history = model.fit(img_train, label_train, epochs = 50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']

best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1

print('Best epoch: %d' % (best_epoch))

Epoch 1/50
1500/1500 [==============================] - 11s 7ms/step - loss: 0.4930 - accuracy: 0.8259 - val_loss: 0.3992 - val_accuracy: 0.8590
Epoch 2/50
1500/1500 [==============================] - 9s 6ms/step - loss: 0.3686 - accuracy: 0.8659 - val_loss: 0.3478 - val_accuracy: 0.8717
Epoch 3/50
1500/1500 [==============================] - 8s 6ms/step - loss: 0.3337 - accuracy: 0.8771 - val_loss: 0.4192 - val_accuracy: 0.8462
Epoch 4/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.3062 - accuracy: 0.8876 - val_loss: 0.3412 - val_accuracy: 0.8767
Epoch 5/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2856 - accuracy: 0.8935 - val_loss: 0.3250 - val_accuracy: 0.8838
Epoch 6/50
1500/1500 [==============================] - 8s 6ms/step - loss: 0.2706 - accuracy: 0.8996 - val_loss: 0.3111 - val_accuracy: 0.8889
Epoch 7/50
1500/1500 [==============================] - 8s 6ms/step - loss: 0.2565 - accuracy: 0.9047 - val_loss: 0.3268 - val_accuracy

实例化超模型并使用上面的最佳周期数对其进行训练

In [17]:
hypermodel = tuner.hypermodel.build(best_hps)

hypermodel.fit(img_train, label_train, epochs = best_epoch, validation_split=0.2)

Epoch 1/34
1500/1500 [==============================] - 10s 6ms/step - loss: 0.4920 - accuracy: 0.8248 - val_loss: 0.4086 - val_accuracy: 0.8512
Epoch 2/34
1500/1500 [==============================] - 8s 5ms/step - loss: 0.3676 - accuracy: 0.8656 - val_loss: 0.3737 - val_accuracy: 0.8678
Epoch 3/34
1500/1500 [==============================] - 8s 5ms/step - loss: 0.3322 - accuracy: 0.8786 - val_loss: 0.3458 - val_accuracy: 0.8728
Epoch 4/34
1500/1500 [==============================] - 8s 5ms/step - loss: 0.3075 - accuracy: 0.8857 - val_loss: 0.3412 - val_accuracy: 0.8747
Epoch 5/34
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2845 - accuracy: 0.8945 - val_loss: 0.3298 - val_accuracy: 0.8808
Epoch 6/34
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2733 - accuracy: 0.8994 - val_loss: 0.3285 - val_accuracy: 0.8832
Epoch 7/34
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2571 - accuracy: 0.9036 - val_loss: 0.3184 - val_accuracy

要完成本教程请在测试数据上评估超模型

In [18]:
eval_result = hypermodel.evaluate(img_test, label_test)

print("[test loss, test accuracy:", eval_result)

313/313 [==============================] - 0s 1ms/step - loss: 0.4712 - accuracy: 0.8804
[test loss, test accuracy: [0.47123461961746216, 0.8804000020027161]


my_dir/intro_to_kt 目录中包含了在超参数搜索期间每次试验（模型配置）运行的详细日志和检查点。如果重新运行超参数搜索，Keras Tuner 将使用这些日志中记录的现有状态来继续搜索。要停用此行为，请在实例化调节器时传递一个附加的 overwrite = True 参数。